In [1]:
import pandas as pd
import datetime as dt
import plotly.graph_objects as go
from technicals.indicators import *
from technicals.patterns import apply_patterns
from guruTester import GuruTester,GuruTester2
from plotting import CandlePlot
import MetaTrader5 as mt5
from datetime import datetime
from getCandle import Rates
from sklearn.neural_network import MLPClassifier
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler  

In [14]:
pair = "EURUSD"
NB_H1_CANDLES = 10000

In [15]:
rates = Rates(pair, NB_H1_CANDLES, mt5.TIMEFRAME_H1)
SPREAD = rates.get_spread()
df_an = rates.get_rates_from_now()
df_an.drop(NB_H1_CANDLES-1,inplace=True) 
df_an.drop(['tick_volume', 'spread', 'real_volume'], axis=1, inplace=True)

In [16]:
rates = Rates(pair, 13*2000, mt5.TIMEFRAME_M5)
df_m5 = rates.get_rates_from_now()
df_an.reset_index(drop=True, inplace=True)
df_m5.reset_index(drop=True, inplace=True)

In [17]:
candle_patterns = ['HANGING_MAN', 'SHOOTING_STAR',
       'SPINNING_TOP', 'MARUBOZU', 'ENGULFING', 'TWEEZER_TOP',
       'TWEEZER_BOTTOM', 'MORNING_STAR', 'EVENING_STAR']

df_an = apply_patterns(df_an)
for cp in candle_patterns:
    df_an[cp] = df_an[cp].astype(float)
df_an.head()

,time,open,high,low,close,body_lower,body_upper,body_bottom_perc,body_top_perc,body_perc,...,body_perc_prev_2,HANGING_MAN,SHOOTING_STAR,SPINNING_TOP,MARUBOZU,ENGULFING,TWEEZER_TOP,TWEEZER_BOTTOM,MORNING_STAR,EVENING_STAR
0,2021-08-11 21:00:00,1.17460,1.17465,1.17370,1.17371,1.17371,1.17460,1.052632,94.736842,93.684211,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2021-08-11 22:00:00,1.17372,1.17413,1.17366,1.17410,1.17372,1.17410,12.765957,93.617021,80.851064,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2021-08-11 23:00:00,1.17411,1.17412,1.17378,1.17396,1.17396,1.17411,52.941176,97.058824,44.117647,...,93.684211,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2021-08-12 00:00:00,1.17351,1.17396,1.17351,1.17382,1.17351,1.17382,0.000000,68.888889,68.888889,...,80.851064,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,2021-08-12 01:00:00,1.17383,1.17411,1.17373,1.17404,1.17383,1.17404,26.315789,81.578947,55.263158,...,44.117647,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
df_an = RSI(df_an)
df_an = BollingerBands(df_an)
df_an = ATR(df_an)
df_an = KeltnerChannels(df_an)
df_an = MACD(df_an)
df_an

,time,open,high,low,close,body_lower,body_upper,body_bottom_perc,body_top_perc,body_perc,...,BB_MA,BB_UP,BB_LW,ATR_14,EMA,KeUp,KeLo,MACD,SIGNAL_MACD,HIST
0,2021-08-11 21:00:00,1.17460,1.17465,1.17370,1.17371,1.17371,1.17460,1.052632,94.736842,93.684211,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-08-11 22:00:00,1.17372,1.17413,1.17366,1.17410,1.17372,1.17410,12.765957,93.617021,80.851064,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-08-11 23:00:00,1.17411,1.17412,1.17378,1.17396,1.17396,1.17411,52.941176,97.058824,44.117647,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-08-12 00:00:00,1.17351,1.17396,1.17351,1.17382,1.17351,1.17382,0.000000,68.888889,68.888889,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-08-12 01:00:00,1.17383,1.17411,1.17373,1.17404,1.17383,1.17404,26.315789,81.578947,55.263158,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,2023-03-22 16:00:00,1.07815,1.07984,1.07757,1.07952,1.07815,1.07952,25.550661,85.903084,60.352423,...,1.077531,1.079318,1.075744,0.001281,1.077588,1.080572,1.074604,0.001096,0.001103,-0.000007
9995,2023-03-22 17:00:00,1.07952,1.07957,1.07835,1.07914,1.07914,1.07952,64.754098,95.901639,31.147541,...,1.077638,1.079515,1.075761,0.001297,1.077735,1.080801,1.074669,0.001091,0.001100,-0.000010
9996,2023-03-22 18:00:00,1.07913,1.08004,1.07885,1.07990,1.07913,1.07990,23.529412,88.235294,64.705882,...,1.077773,1.079808,1.075738,0.001311,1.077942,1.081036,1.074848,0.001134,0.001107,0.000027
9997,2023-03-22 19:00:00,1.07991,1.07998,1.07898,1.07944,1.07944,1.07991,46.000000,93.000000,47.000000,...,1.077905,1.080022,1.075788,0.001338,1.078084,1.081122,1.075046,0.001119,0.001110,0.000009


In [19]:

df_an

,time,open,high,low,close,body_lower,body_upper,body_bottom_perc,body_top_perc,body_perc,...,BB_MA,BB_UP,BB_LW,ATR_14,EMA,KeUp,KeLo,MACD,SIGNAL_MACD,HIST
0,2021-08-11 21:00:00,1.17460,1.17465,1.17370,1.17371,1.17371,1.17460,1.052632,94.736842,93.684211,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-08-11 22:00:00,1.17372,1.17413,1.17366,1.17410,1.17372,1.17410,12.765957,93.617021,80.851064,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-08-11 23:00:00,1.17411,1.17412,1.17378,1.17396,1.17396,1.17411,52.941176,97.058824,44.117647,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-08-12 00:00:00,1.17351,1.17396,1.17351,1.17382,1.17351,1.17382,0.000000,68.888889,68.888889,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-08-12 01:00:00,1.17383,1.17411,1.17373,1.17404,1.17383,1.17404,26.315789,81.578947,55.263158,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,2023-03-22 16:00:00,1.07815,1.07984,1.07757,1.07952,1.07815,1.07952,25.550661,85.903084,60.352423,...,1.077531,1.079318,1.075744,0.001281,1.077588,1.080572,1.074604,0.001096,0.001103,-0.000007
9995,2023-03-22 17:00:00,1.07952,1.07957,1.07835,1.07914,1.07914,1.07952,64.754098,95.901639,31.147541,...,1.077638,1.079515,1.075761,0.001297,1.077735,1.080801,1.074669,0.001091,0.001100,-0.000010
9996,2023-03-22 18:00:00,1.07913,1.08004,1.07885,1.07990,1.07913,1.07990,23.529412,88.235294,64.705882,...,1.077773,1.079808,1.075738,0.001311,1.077942,1.081036,1.074848,0.001134,0.001107,0.000027
9997,2023-03-22 19:00:00,1.07991,1.07998,1.07898,1.07944,1.07944,1.07991,46.000000,93.000000,47.000000,...,1.077905,1.080022,1.075788,0.001338,1.078084,1.081122,1.075046,0.001119,0.001110,0.000009


In [20]:
SPREAD = (SPREAD//1e-5)*1e-5 # todo: do it right!!!
SPREAD # 0

5e-05

In [21]:
df_an['low_perv'] = df_an.low.shift(1)
df_an['high_perv'] = df_an.high.shift(1)
df_an['low_next'] = df_an.low.shift(-1)
df_an['high_next'] = df_an.high.shift(-1)
df_an.pivots_l = (df_an.low_perv > df_an.low) & (df_an.low_next > df_an.low)
df_an.pivots_h = (df_an.high_perv < df_an.high) & (df_an.high_next < df_an.high)
df_an.dropna(inplace=True)
df_an.reset_index(drop=True, inplace=True)
df_an

C:\Users\meysam\AppData\Local\Temp\ipykernel_9512\3875528746.py:5: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df_an.pivots_l = (df_an.low_perv > df_an.low) & (df_an.low_next > df_an.low)
C:\Users\meysam\AppData\Local\Temp\ipykernel_9512\3875528746.py:6: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df_an.pivots_h = (df_an.high_perv < df_an.high) & (df_an.high_next < df_an.high)


,time,open,high,low,close,body_lower,body_upper,body_bottom_perc,body_top_perc,body_perc,...,EMA,KeUp,KeLo,MACD,SIGNAL_MACD,HIST,low_perv,high_perv,low_next,high_next
0,2021-08-13 06:00:00,1.17396,1.17404,1.17368,1.17384,1.17384,1.17396,44.444444,77.777778,33.333333,...,1.173556,1.174608,1.172504,-8.265582e-07,-0.000079,0.000078,1.17377,1.17413,1.17371,1.17398
1,2021-08-13 07:00:00,1.17384,1.17398,1.17371,1.17378,1.17378,1.17384,25.925926,48.148148,22.222222,...,1.173578,1.174508,1.172648,1.521020e-05,-0.000058,0.000073,1.17368,1.17404,1.17348,1.17384
2,2021-08-13 08:00:00,1.17377,1.17384,1.17348,1.17356,1.17356,1.17377,22.222222,80.555556,58.333333,...,1.173576,1.174434,1.172718,1.103528e-05,-0.000043,0.000054,1.17371,1.17398,1.17307,1.17374
3,2021-08-13 09:00:00,1.17356,1.17374,1.17307,1.17318,1.17318,1.17356,16.417910,73.134328,56.716418,...,1.173537,1.174393,1.172681,-2.105990e-05,-0.000038,0.000017,1.17348,1.17384,1.17290,1.17374
4,2021-08-13 10:00:00,1.17320,1.17374,1.17290,1.17333,1.17320,1.17333,35.714286,51.190476,15.476190,...,1.173517,1.174479,1.172555,-3.455641e-05,-0.000037,0.000003,1.17307,1.17374,1.17330,1.17417
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9960,2023-03-22 15:00:00,1.07818,1.07894,1.07759,1.07814,1.07814,1.07818,40.740741,43.703704,2.962963,...,1.077384,1.079994,1.074774,1.048660e-03,0.001105,-0.000056,1.07790,1.08001,1.07757,1.07984
9961,2023-03-22 16:00:00,1.07815,1.07984,1.07757,1.07952,1.07815,1.07952,25.550661,85.903084,60.352423,...,1.077588,1.080572,1.074604,1.096123e-03,0.001103,-0.000007,1.07759,1.07894,1.07835,1.07957
9962,2023-03-22 17:00:00,1.07952,1.07957,1.07835,1.07914,1.07914,1.07952,64.754098,95.901639,31.147541,...,1.077735,1.080801,1.074669,1.090503e-03,0.001100,-0.000010,1.07757,1.07984,1.07885,1.08004
9963,2023-03-22 18:00:00,1.07913,1.08004,1.07885,1.07990,1.07913,1.07990,23.529412,88.235294,64.705882,...,1.077942,1.081036,1.074848,1.134300e-03,0.001107,0.000027,1.07835,1.07957,1.07898,1.07998


In [22]:
BUY = 1
SELL = -1
NONE = 0

df_an['BUY_SIG'] = 0
df_an['SELL_SIG'] = 0
df_an['SIGNAL_LABEL'] = 0
df_an['SIGNAL'] = 0
df_an['TP'] = 0
df_an['SL'] = 0

In [23]:
for i in range(1,len(df_an)):
    if df_an.pivots_l[i-1]:
        for j in range(i,len(df_an)):
            if( df_an.low[j] <= df_an.low[i-1] ):
                break
            else:
                if(df_an.high[j] > 2*(df_an.close[i] - df_an.low[i-1] ) + df_an.close[i]):
                    df_an.BUY_SIG[i] = 1
                    df_an.SIGNAL[i] = BUY
                    df_an.SIGNAL_LABEL[i] = BUY
                    df_an['SL'][i] = df_an.low[i-1] - SPREAD
                    df_an['TP'][i] = 2*(df_an.close[i] - df_an.low[i-1] ) + df_an.close[i] #+ SPREAD
                    break
    if df_an.pivots_h[i-1]:
        for j in range(i,len(df_an)):
            if( df_an.high[j] >= df_an.high[i-1] ):
                break
            else:
                if(df_an.low[j] < 2*(df_an.close[i] - df_an.high[i-1] ) + df_an.close[i]):
                    df_an.SELL_SIG[i] = 1
                    df_an.SIGNAL[i] = SELL
                    df_an.SIGNAL_LABEL[i] = SELL
                    df_an['SL'][i] = df_an.high[i-1] + SPREAD
                    df_an['TP'][i] = 2*(df_an.close[i] - df_an.high[i-1] ) + df_an.close[i]
                    break

C:\Users\meysam\AppData\Local\Temp\ipykernel_9512\3433021398.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_an.BUY_SIG[i] = 1
C:\Users\meysam\AppData\Local\Temp\ipykernel_9512\3433021398.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_an.SIGNAL[i] = BUY
C:\Users\meysam\AppData\Local\Temp\ipykernel_9512\3433021398.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_an.SIGNAL_LABEL[i] = BUY
C:\Users\meysam\AppData\Loca

In [24]:
df_an.to_csv(pair+'.csv')

In [ ]:
# pd.read_csv(pair+'.csv')

In [ ]:
df_an

In [12]:
X = []
Y = [] 

feature_col = [ 'open', 'high', 'low', 'close', 'RSI_14', 'BB_MA', 'BB_UP',
       'BB_LW', 'ATR_14', 'EMA', 'KeUp', 'KeLo', 'MACD', 'SIGNAL_MACD', 'HIST',
       'body_lower', 'body_upper', 'body_bottom_perc', 'body_top_perc',
       'body_perc', 'direction', 'body_size', 'low_change', 'high_change', 'mid_point', 'mid_point_prev_2', 'body_size_prev',
       'direction_prev', 'direction_prev_2', 'body_perc_prev',
       'body_perc_prev_2', 'HANGING_MAN', 'SHOOTING_STAR', 'SPINNING_TOP',
       'MARUBOZU', 'ENGULFING', 'TWEEZER_TOP', 'TWEEZER_BOTTOM',
       'MORNING_STAR', 'EVENING_STAR']
for i in range(len(df_an)):
    feature = []
    for fc in feature_col:
        feature.append(df_an[fc][i])
    X.append(feature)
    Y.append(df_an.SIGNAL_LABEL[i])

In [ ]:
len(feature_col)

In [ ]:
X_train = X[:8000]
Y_train = Y[:8000]
X_test = X[8000:]
Y_test = Y[8000:]

In [ ]:
help(MLPClassifier)

In [ ]:
scaler = StandardScaler()  
scaler.fit(X_train)  
X_train = scaler.transform(X_train)  
# >>> # apply same transformation to test data
# >>> X_test = scaler.transform(X_test)  

In [ ]:
clf = MLPClassifier( solver='adam', alpha=0,learning_rate_init=1e-5,
                     tol=0.0000010, hidden_layer_sizes=(50,40), 
                    random_state=1,max_iter=100000,verbose=True)
# clf = MLPClassifier(learning_rate_init=1e-6, warm_start = True, random_state = 2,hidden_layer_sizes= (60,40),max_iter=100000,verbose=True, tol=0.00000010,n_iter_no_change=40)

In [ ]:
clf.fit(X_train, Y_train)

In [ ]:
y_perdict = clf.predict(X_test)

In [ ]:
confusion_matrix(Y_test, y_perdict)

In [ ]:
Y_test

In [ ]:
y_perdict = list(y_perdict)

In [ ]:
len(y_perdict)

In [ ]:
df_test = df_an.iloc[8000:].copy()
df_test.reset_index(drop=True, inplace=True)
df_test["SIGNAL"] = y_perdict

In [ ]:
df_test.SIGNAL

In [ ]:
gt = GuruTester2(
        df_test,
        df_m5,
        SPREAD,
        use_spread=True
    )

In [ ]:
df_res_m5 = gt.run_test()

In [ ]:
df_res_m5['balance'] = 100
loss_margin = 0.03
perv_balance = 100

# for index, row in df_res_m5.iterrows():
for i in range(len(df_res_m5)):# start from zero!!!!!!!!!!!!!!!!!!!!!
    if(df_res_m5.result[i] == -1):
        df_res_m5['balance'][i] = perv_balance*(1-loss_margin) 
    elif(df_res_m5.result[i] == 2):
        profit_to_loss = abs( (df_res_m5['start_price'][i]-df_res_m5['TP'][i])/((df_res_m5['start_price'][i]-df_res_m5['SL'][i])) )
        df_res_m5['balance'][i] = perv_balance*(1+profit_to_loss*loss_margin) 
    perv_balance = df_res_m5['balance'][i]

In [ ]:
df_res_m5.result.value_counts()

In [ ]:
df_res_m5_plot = df_res_m5#.loc[:100]
df_res_m5_plot['time'] = df_res_m5_plot['end_time']
cp = CandlePlot(df_res_m5_plot, candles=False)
cp.show_plot(line_traces=['balance'])